# Compute metrics on toy data

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from pathlib import Path

import pandas as pd

from src.dependence_measures.compare import compute_bivariate_scores

In [ ]:
PROCESSED_DATA_FOLDER_PATH = Path("../data/processed")

INPUTS_COLUMNS = ["x_0"]
OUTPUTS_COLUMNS = ["y"]

## Load noiseless toy data

In [ ]:
data_df = pd.read_csv(PROCESSED_DATA_FOLDER_PATH.joinpath("toy_data", "noiseless", "cubic_500.csv"), index_col=False)
data_df.head()

## Compute bi-variate metrics

In [ ]:
compute_bivariate_scores(data_df, ["0"], ["1"])

In [ ]:
scores_list = []

for file in sorted(PROCESSED_DATA_FOLDER_PATH.joinpath("toy_data", "noiseless").glob("*.csv")):

    data_df = pd.read_csv(file, index_col=False)

    scores_df = compute_bivariate_scores(data_df, ["0"], ["1"])
    scores_df["file"] = file.name

    scores_list.append(scores_df)

scores_df = pd.concat(scores_list)

In [ ]:
scores_df

In [ ]:
import numpy as np
import math

N_BINS = 9

x = np.array(data_df["0"])
y = np.array(data_df["1"])

Pxy = np.histogram2d(x, y, bins=N_BINS)[0].T
Pxy = Pxy / Pxy.sum()
Px = Pxy.sum(axis=0)
Py = Pxy.sum(axis=1)

B = Pxy * (1 / np.sqrt(Px)) * (1 / np.sqrt(Py))

B = np.nan_to_num(B, nan=0, posinf=0, neginf=0)

U, S, V = np.linalg.svd(B)

S[1]

In [ ]:
type(B)